Steps and methods covered in this notebook
- Read the main dataset
- Drop the unnecessary columns
- Drop the rows having more than 2 null values in the remaining columns
</n>
- Implement the MICE and Autoencoders imputation
- Then use Cluster-based SMOTE (KMeans SMOTE) for both the Mice and AE datasets
</n>
- Before last step - group the columns under thier headers
- As a last step implement PCA for both the datasets and save the datasets as excel files separately


(Refer the Preprocessing_PCA_SMOTE.ipynb for the lines of code)